In [1]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import re
from datetime import datetime

In [2]:
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

In [3]:
class Preprocesser:
    def __init__(self):
        self.kospi = ""
        self.kosdaq = ""
        # need kospi file
        # MK Embedding file
    
        #papago 번역 함수
    
    
    #폴더에 있는 파일 리스트 가져오기
    def get_filelist(self):
        filelist = os.listdir(os.getcwd())
        return filelist


    #파일 불러오기
    def load_funda_data(self, data_name):
        data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')
        return data    

    def load_feature_data(self, data_name):
        if 'csv' in data_name:
            data = pd.read_csv("{}".format(data_name), thousands=',')
        else:
            data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')


    # 재무 데이터 컬럼 이름 설정
    def set_col_names(self, df):
        col_list = ['Date', 'asset(1000)', 'debt(1000)', 'sales(1000)',
                    'adjusted_price', 'profit_rate', 'volumne(mean_5d)(week)',
                    'volumne(week)','profit(1000)',
                    'term_profit(1000)', 'gross_margin(1000)','adjusted_BPS', 'adjusted_EPS']
        df.columns = col_list
        return df
    
    def add_feature_name(self, feature_name_list, file_name):
        dotpos = file_name.find('.')
        file_name = file_name[:dotpos]
        new_col = ['Date']
        for i in range(feature_name_list)-1:
            new_name = feature_name_list[i+1]+file_name
            new_col += new_name
        return new_col


#     # 피쳐 데이터 컬럼 이름 설정(2차전지) (나머지 산업에 대해서도 필요)
#     def feature_col_name(df, name):
#         """
#         name : dubai, nickel, mirenal, wti, lithum, cobalt_l,cobalt_r, brent

#         """
#         if name =='dubai':
#             col_list = dubai_price_col_list = ['Date', 'dubai_price','dubai_price_change','dubai_price_change_rate','dubai_stock']
#         elif name == 'nickel':
#             col_list = nickel_price_col_list = ['Date', 'nickel_price','nickel_price_change','nickel_price_change_rate','nikcel_stock']
#         elif name == 'mineral':
#             col_list = mineral_col_list = ['Date', 'mineral_index','mineral_index_change','mineral_index_change_rate']
#         elif name == 'wti': 
#             col_list = wti_price_col_list = ['Date', 'wti_price','wti_price_change','wti_price_change_rate','wti_stock']
#         elif name == 'lithum':
#             col_list = lithum_price_col_list = ['Date', 'lithum_price','lithum_price_change','lithum_price_change_rate','lithum_stock']
#         elif name == 'cobalt_l': 
#             col_list = cobalt_l_price_col_list = ['Date', 'cobalt_l_price','coblat_l_price_change','cobalt_l_price_change_rate','cobalt_l_stock']
#         elif name == 'cobalt_r': 
#             col_list = cobalt_r_price_col_list = ['Date', 'cobalt_r_price','coblat_r_price_change','cobalt_r_price_change_rate','cobalt_r_stock']
#         elif name == 'brent': 
#             col_list = brent_price_col_list = ['Date', 'brent_price','brent_price_change','brent_price_change_rate','brent_stock']
        
#         df.columns = col_list

#         return df


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(재무)
    def set_funda_data(self, data, start_date):

        """
        set_funda_data(df,'2010-01-05')
        """
        data = data.iloc[5:]

        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(피쳐)
    def set_feature_data(self, data, start_date):

        """
        set_funda_data('samsung_elec",'2010-01-05')
        """
        
        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data



    #화장품 fundamental data 내 존재하는 monthly data를 날짜별로 만들기  ( 현재 datetime으로 되어 있고, 모든 날짜에 값 부여 필요 )
    def mkdatetime_mth(self, data):
        data2 = data[data['년월']['년월'].str.contains("월")]
        index = data2.index
        for i, mth in enumerate(data2['년월']['년월']):
            m = int(mth[1:-1])
            idx = index[i]
            idx = idx - m
            y = int(data['년월']['년월'][idx][:-1])
            print(y, m)

            data2.iat[i, 0] = datetime(y, m, 1)

        return data2






    # #데이터 불러와서 datetime으로 만들고, 각 컬럼 모두 합치기, 수연이한테 각 컬럼명이 무엇을 뜻하는 지 물어볼 것
    # def read_mth_data(self, data, domain):
    #     domain = "화장품"
    #     os.chdir(r"G:\공유 드라이브\Boad ADV Stock\{}\fundamental".format(domain))
        
    #     data = pd.read_excel("330112_오렌지유.xls", skiprows = [0, 1], header=[0,1])
    #     pd.read_excel("330113_레몬유.xls", skiprows = [0, 1], header=[0,1])



    # 특정 연도 사이 휴일 리스트
    def check_holiday(self, start_year, last_year):
        
        """
        check_hodiday(2010,2020)
        """

        total_holiday = list()
        for year in range(start_year, last_year):
            hlist = list()
            krholiday = [d.strftime('%Y-%m-%d') for d in pytimekr.holidays(year)]
            week_sat = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SAT')]
            week_sun = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SUN')]
            hlist.extend(krholiday)
            hlist.extend(week_sat)
            hlist.extend(week_sun)
            total_holiday.extend(pd.unique(hlist))
        sorted(total_holiday)
        return total_holiday


    # 특정 연도 사이 휴일 x 리스트
    def check_workday(self, start_year, last_year):
        """
        check_workday(2010,2020)
        """

        workday = list()
        for d in pd.date_range('{}-01-01'.format(start_year),'{}-12-31'.format(last_year)):
            if d.strftime('%Y-%m-%d') not in check_holiday(start_year, last_year):
                workday.append(d.strftime('%Y-%m-%d'))       
        return workday


    # 데이터 프레임에서 휴일 아닌 날만 뽑기
    def set_index_workday(self, df,start_year,last_year):
        work = pd.DataFrame(check_workday(start_year, last_year),columns=['Date'])
        work.Date =  pd.to_datetime(work.Date)
        df.reset_index(inplace=True)
        work_df = pd.merge(df,work, on ='Date',how='inner')
        work_df.set_index(['Date'],inplace=True)
        # work_df = pd.concat([df,work], join_axes=['Date'],join='inner',axis=0)

        return work_df

    # 회시별 재무 데이터 테이블
    def individual_feauture(self, stockname):
        df = load_funda_data(stockname)
        df = set_col_names(df)
        df = set_funda_data(df)
        return df

    # 산업 공통 피쳐
    def common_feature(self,featurename):
        
        df = load_funda_data(featurename)
        df = feature_col_name(df)
        df = set_feature_data(df)
        return df

    # 산업 공통 피쳐 테이블
    def common_feature_table(self):
        os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지\funda_data')
        filelist = os.listdir(os.getcwd())
        # filelist
        feature_df = pd.DataFrame()
        for feature in filelist:
            add_df = common_feature(feature)
            feature_df = pd.merge(feature_df, add_df, on='Date', how ='left')
        return


In [4]:
class battery_preprocesser(Preprocesser):
    
    def get_filelist(self, folder):
        #folder = 'fn_data' , 'tech_data'
        filelist = os.listdir(os.chdir(r'G:\공유 드라이브\Boad ADV Stock\{}\{}'.format(industry,folder)))
        return filelist

In [5]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지')

In [6]:
os.getcwd()

'G:\\공유 드라이브\\Boad ADV Stock\\2차전지'

In [7]:
os.chdir('./fn_data')

In [8]:
battery_preprocesser = battery_preprocesser()

In [9]:
industry = '2차전지'
battery_preprocesser.get_filelist('fn_data')

['피앤이솔루션.xlsx', '엘앤에프.xlsx', '이엔에프테크놀로지.xlsx', '코스모신소재.xlsx', '씨아이에스.xlsx']

In [10]:
data  = battery_preprocesser.load_funda_data(battery_preprocesser.get_filelist('fn_data')[0])

In [11]:
company = '씨아이에스'
data = pd.read_excel('{}.xlsx'.format(company))

In [12]:
data = battery_preprocesser.set_col_names(data)
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,Symbol Name,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스,씨아이에스
1,Kind,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),SSC,SSC,SSC,SSC,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),NVAL-IFRS(C),NVAL-IFRS(C)
2,Item,6000901001,6000902001,6000904001,S410000700,S41000170F,S41000620F,S41000080F,6000906001,6000908001,6000904007,6000601036,6000601003
3,Item Name,총자산(천원),총부채(천원),매출액(천원),수정주가(원),수익률(%),거래량 (5일 평균)(주),거래량(주),영업이익(천원),당기순이익(천원),매출총이익(천원),수정BPS(원),수정EPS(원)
4,Frequency,ANNUAL,ANNUAL,ANNUAL,DAILY,DAILY,DAILY,DAILY,ANNUAL,ANNUAL,ANNUAL,ANNUAL,ANNUAL


In [13]:
data = battery_preprocesser.set_funda_data(data , '2010-01-11')
data.head()

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
Date,,,,,,,,,,,,
2010-01-11,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
2010-01-12,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
2010-01-13,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
2010-01-14,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
2010-01-15,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32


In [14]:
data = data.reset_index()
data.replace('-',np.nan, inplace=True)
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,2010-01-11,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
1,2010-01-12,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
2,2010-01-13,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
3,2010-01-14,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32
4,2010-01-15,24045185,14580313,28735454,NaN,NaN,NaN,NaN,6836614,5891293,8727267,222.23,138.32


In [15]:
data.Date = pd.to_datetime(data.Date)

# feature data

In [16]:
battery_preprocesser.get_filelist('feature_data')

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [17]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

# 니켈

In [18]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [19]:
nickel_name = battery_preprocesser.get_filelist('feature_data')[1]
nickel_name

'니켈가격.xlsx'

In [20]:
nickel = pd.read_excel("{}".format(nickel_name), thousands=',')
# nickel.head(10)

In [21]:
nickel.columns = ['Date','price','change','%change','stock']
nickel.columns = add_feature_name(nickel.columns,'nickel')
# nickel.head()

In [22]:
nickel.Date = pd.to_datetime(nickel.Date)

In [23]:
nickel.head()

,Date,price_nickel,change_nickel,%change_nickel,stock_nickel
0,2010-01-04,18855,375,2.03,158424
1,2010-01-05,18615,-240,-1.27,158424
2,2010-01-06,18955,340,1.83,158388
3,2010-01-07,18750,-205,-1.08,158814
4,2010-01-08,18125,-625,-3.33,158982


# 광물종합

In [24]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [25]:
mineral_index_name = battery_preprocesser.get_filelist('feature_data')[0]
mineral_index_name

'광물종합지수.xls'

In [26]:
mineral_index_1 = pd.read_excel("{}".format(mineral_index_name), thousands=',', sheet_name='광물종합지수')
mineral_index_2 = pd.read_excel("{}".format(mineral_index_name), thousands=',', sheet_name='메이저금속지수')
mineral_index = pd.merge(mineral_index_1,mineral_index_2, on = '기준일',how='outer')

In [27]:
mineral_index.head(10)

,기준일,광물종합지수,주등락가_x,주등락율_x,메이저금속지수,주등락가_y,주등락율_y
0,20110104,2163.53,0.00,0.00,2171.91,0.00,0.00
1,20110105,2176.60,13.07,0.60,2180.38,8.47,0.39
2,20110106,2169.06,-7.54,-0.35,2153.97,-26.41,-1.21
3,20110110,2211.83,20.69,0.94,2170.66,-7.38,-0.34
4,20110111,2212.86,1.03,0.05,2159.69,-10.97,-0.51
5,20110112,2233.33,20.48,0.93,2178.35,18.66,0.86
6,20110113,2248.30,14.97,0.67,2196.62,18.27,0.84
7,20110114,2243.99,-4.31,-0.19,2203.71,7.09,0.32
8,20110117,2246.15,2.16,0.10,2192.67,-11.04,-0.50
9,20110118,2259.71,13.56,0.60,2210.10,17.43,0.79


In [28]:
mineral_index.columns = ['Date','index','change','%change','major','major_change','major_%chagne']
mineral_index.columns = add_feature_name(mineral_index.columns,'mineral')
mineral_index.head()

,Date,index_mineral,change_mineral,%change_mineral,major_mineral,major_change_mineral,major_%chagne_mineral
0,20110104,2163.53,0.00,0.00,2171.91,0.00,0.00
1,20110105,2176.60,13.07,0.60,2180.38,8.47,0.39
2,20110106,2169.06,-7.54,-0.35,2153.97,-26.41,-1.21
3,20110110,2211.83,20.69,0.94,2170.66,-7.38,-0.34
4,20110111,2212.86,1.03,0.05,2159.69,-10.97,-0.51


In [29]:
mineral_index.Date = pd.to_datetime(mineral_index.Date,format='%Y%m%d')
mineral_index.head()

,Date,index_mineral,change_mineral,%change_mineral,major_mineral,major_change_mineral,major_%chagne_mineral
0,2011-01-04,2163.53,0.00,0.00,2171.91,0.00,0.00
1,2011-01-05,2176.60,13.07,0.60,2180.38,8.47,0.39
2,2011-01-06,2169.06,-7.54,-0.35,2153.97,-26.41,-1.21
3,2011-01-10,2211.83,20.69,0.94,2170.66,-7.38,-0.34
4,2011-01-11,2212.86,1.03,0.05,2159.69,-10.97,-0.51


# 탄산리튬

In [30]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [31]:
co_lithum_name = battery_preprocesser.get_filelist('feature_data')[2]
co_lithum_name

'탄산리튬가격.xlsx'

In [32]:
co_lithum = pd.read_excel("{}".format(co_lithum_name), thousands=',')
co_lithum.head()

,기준일,기준가격,등락가,등락비율,재고량
0,2010-02-01,36.0,0.0,0.0,0
1,2010-02-03,36.0,0.0,0.0,0
2,2010-02-05,36.0,0.0,0.0,0
3,2010-02-08,36.0,0.0,0.0,0
4,2010-02-10,36.0,0.0,0.0,0


In [33]:
co_lithum.columns = ['Date','price','change','%change','stock']
co_lithum.columns = add_feature_name(co_lithum.columns,'co_lithum')
co_lithum= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]
co_lithum.head()

,Date,price_co_lithum,change_co_lithum,%change_co_lithum
0,2010-02-01,36.0,0.0,0.0
1,2010-02-03,36.0,0.0,0.0
2,2010-02-05,36.0,0.0,0.0
3,2010-02-08,36.0,0.0,0.0
4,2010-02-10,36.0,0.0,0.0


In [34]:
co_lithum.Date = pd.to_datetime(co_lithum.Date)
co_lithum.head()

,Date,price_co_lithum,change_co_lithum,%change_co_lithum
0,2010-02-01,36.0,0.0,0.0
1,2010-02-03,36.0,0.0,0.0
2,2010-02-05,36.0,0.0,0.0
3,2010-02-08,36.0,0.0,0.0
4,2010-02-10,36.0,0.0,0.0


# 코발트(메탈LME)

In [35]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [36]:
cobalt_name = battery_preprocesser.get_filelist('feature_data')[3]
cobalt_name

'코발트가격(메탈LME).xlsx'

In [37]:
cobalt_LME = pd.read_excel("{}".format(cobalt_name), thousands=',')
cobalt_LME.head()

,기준일,기준가격,등락가,등락비율,재고량
0,2010-05-19,41000.0,0.0,0.00,0
1,2010-05-20,39500.0,-1500.0,-3.66,0
2,2010-05-21,39500.0,0.0,0.00,0
3,2010-05-24,40500.0,1000.0,2.53,0
4,2010-05-25,41800.0,1300.0,3.21,0


In [38]:
cobalt_LME.columns = ['Date','price','change','%change','stock']
cobalt_LME.columns = add_feature_name(cobalt_LME.columns,'cobalt_LME')
cobalt_LME = cobalt_LME.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]
cobalt_LME.head()

,Date,price_cobalt_LME,change_cobalt_LME,%change_cobalt_LME
0,2010-05-19,41000.0,0.0,0.00
1,2010-05-20,39500.0,-1500.0,-3.66
2,2010-05-21,39500.0,0.0,0.00
3,2010-05-24,40500.0,1000.0,2.53
4,2010-05-25,41800.0,1300.0,3.21


In [39]:
cobalt_LME.Date = pd.to_datetime(cobalt_LME.Date)
cobalt_LME.head()

,Date,price_cobalt_LME,change_cobalt_LME,%change_cobalt_LME
0,2010-05-19,41000.0,0.0,0.00
1,2010-05-20,39500.0,-1500.0,-3.66
2,2010-05-21,39500.0,0.0,0.00
3,2010-05-24,40500.0,1000.0,2.53
4,2010-05-25,41800.0,1300.0,3.21


# 코발트(rotterdam)

In [40]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [41]:
cobalt_name = battery_preprocesser.get_filelist('feature_data')[4]
cobalt_name

'코발트가격(rotterdam).xlsx'

In [42]:
cobalt_rotterdam = pd.read_excel("{}".format(cobalt_name), thousands=',')
cobalt_rotterdam.head()

,기준일,기준가격,등락가,등락비율,재고량
0,2010-01-01,21.50,0.25,1.18,0
1,2010-01-08,22.00,0.50,2.33,0
2,2010-01-15,23.25,1.25,5.68,0
3,2010-01-22,23.75,0.50,2.15,0
4,2010-01-29,23.19,-0.56,-2.36,0


In [43]:
cobalt_rotterdam.columns = ['Date','price','change','%change','stock']
cobalt_rotterdam.columns = add_feature_name(cobalt_rotterdam.columns,'cobalt_rotterdam')
cobalt_rotterdam = cobalt_rotterdam.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]
cobalt_rotterdam.head()

,Date,price_cobalt_rotterdam,change_cobalt_rotterdam,%change_cobalt_rotterdam
0,2010-01-01,21.50,0.25,1.18
1,2010-01-08,22.00,0.50,2.33
2,2010-01-15,23.25,1.25,5.68
3,2010-01-22,23.75,0.50,2.15
4,2010-01-29,23.19,-0.56,-2.36


In [44]:
cobalt_rotterdam.Date = pd.to_datetime(cobalt_rotterdam.Date)
cobalt_rotterdam.head()

,Date,price_cobalt_rotterdam,change_cobalt_rotterdam,%change_cobalt_rotterdam
0,2010-01-01,21.50,0.25,1.18
1,2010-01-08,22.00,0.50,2.33
2,2010-01-15,23.25,1.25,5.68
3,2010-01-22,23.75,0.50,2.15
4,2010-01-29,23.19,-0.56,-2.36


# kospi,kosdqt

In [45]:
ko_market_name = battery_preprocesser.get_filelist('feature_data')[5]
ko_market_name

'kospi,kosdaq.xlsx'

In [46]:
ko_market = pd.read_excel("{}".format(ko_market_name), thousands=',')
ko_market.head()

,Symbol Name,코스닥,코스피
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


In [47]:
ko_market.columns = ['Date','kospi','kosdaq']
# ko_market.columns = add_feature_name(cobalt_rotterdam.columns,'cobalt_rotterdam')
# cobalt_rotterdam = cobalt_rotterdam.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]
ko_market.head()

,Date,kospi,kosdaq
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


In [48]:
ko_market.Date = pd.to_datetime(ko_market.Date)
ko_market.head()

,Date,kospi,kosdaq
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


# kospi

In [49]:
# ko_market = battery_preprocesser.get_filelist('feature_data')[6]
# ko_market

# kosdaq

In [50]:
# ko_market = battery_preprocesser.get_filelist('feature_data')[7]
# ko_market

# usd_krw 

In [51]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [52]:
usd_krw_name = battery_preprocesser.get_filelist('feature_data')[9]
usd_krw_name

'USD_KRW 내역.csv'

In [53]:
usd_krw = pd.read_csv("{}".format(usd_krw_name), thousands=',')
usd_krw.head()

,날짜,종가,오픈,고가,저가,변동 %
0,2019년 12월 31일,1155.07,1157.97,1159.19,1153.14,-0.25%
1,2019년 12월 30일,1157.97,1160.97,1161.30,1154.89,-0.15%
2,2019년 12월 27일,1159.68,1162.15,1162.83,1158.12,-0.23%
3,2019년 12월 26일,1162.30,1161.04,1163.42,1160.34,0.13%
4,2019년 12월 25일,1160.76,1163.52,1164.09,1160.68,-0.24%


In [54]:
usd_krw.columns = ['Date','last','open','high','low','%change']
usd_krw.sort_values(by='Date', inplace=True)
usd_krw.columns = add_feature_name(usd_krw.columns,'usd_krw')
# usd_krw = usd_krw.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

usd_krw.head()

,Date,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw
2607,2010년 01월 01일,1166.08,1166.08,1166.08,1166.08,0.00%
2606,2010년 01월 04일,1150.00,1164.07,1164.07,1149.35,-1.38%
2605,2010년 01월 05일,1140.20,1151.00,1151.00,1136.30,-0.85%
2604,2010년 01월 06일,1129.20,1140.20,1144.70,1129.20,-0.96%
2603,2010년 01월 07일,1138.20,1129.20,1138.20,1127.80,0.80%


In [55]:
usd_krw['%change_usd_krw'] = usd_krw['%change_usd_krw'].str.replace('%','')
usd_krw.Date = usd_krw.Date.str.replace('년 ','')
usd_krw.Date = usd_krw.Date.str.replace('월 ','')
usd_krw.Date = usd_krw.Date.str.replace('일','')
usd_krw.Date = pd.to_datetime(usd_krw.Date, format='%Y %m %d')
usd_krw.head()

,Date,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw
2607,2010-01-01,1166.08,1166.08,1166.08,1166.08,0.00
2606,2010-01-04,1150.00,1164.07,1164.07,1149.35,-1.38
2605,2010-01-05,1140.20,1151.00,1151.00,1136.30,-0.85
2604,2010-01-06,1129.20,1140.20,1144.70,1129.20,-0.96
2603,2010-01-07,1138.20,1129.20,1138.20,1127.80,0.80


In [56]:
usd_krw['%change_usd_krw'][1]+usd_krw['%change_usd_krw'][2]

'-0.15-0.23'

# korean_bond

In [57]:
battery_preprocesser.get_filelist('feature_data')[:]

['광물종합지수.xls',
 '니켈가격.xlsx',
 '탄산리튬가격.xlsx',
 '코발트가격(메탈LME).xlsx',
 '코발트가격(rotterdam).xlsx',
 'kospi,kosdaq.xlsx',
 'kospi.xlsx',
 'kosdaq.xlsx',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW 내역.csv',
 'kospi_volatility.csv',
 'WTI유 선물 내역.csv',
 '브렌트유 선물 내역.csv']

In [58]:
korean_bond_name = battery_preprocesser.get_filelist('feature_data')[8]
korean_bond_name

'South Korea 5-Year Bond Yield.csv'

In [59]:
korean_bond = pd.read_csv("{}".format(korean_bond_name), thousands=',')
korean_bond.head()

,Date,Price,Open,High,Low,Change %
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%


In [60]:
korean_bond.columns = ['Date','last','open','high','low','%change']

korean_bond.columns = add_feature_name(korean_bond.columns,'korean_bond')
# usd_krw = usd_krw.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

korean_bond.head()

,Date,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%


In [61]:
korean_bond['%change_korean_bond'] = korean_bond['%change_korean_bond'].str.replace('%','')
korean_bond.Date = korean_bond.Date.str.replace(',','')
korean_bond.Date = pd.to_datetime(korean_bond.Date, format='%b %d %Y')
korean_bond.sort_values(by='Date', inplace=True)
korean_bond.head()

,Date,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond
2977,2010-01-04,4.960,4.960,4.960,4.960,1.12
2976,2010-01-05,4.949,4.949,4.949,4.949,-0.22
2975,2010-01-06,4.880,4.880,4.880,4.880,-1.39
2974,2010-01-07,4.920,4.920,4.920,4.920,0.82
2973,2010-01-08,4.910,4.910,4.910,4.910,-0.20


# kospi_vol

In [62]:
kospi_vol_name = battery_preprocesser.get_filelist('feature_data')[10]
kospi_vol_name

'kospi_volatility.csv'

In [63]:
kospi_vol = pd.read_csv("{}".format(kospi_vol_name), thousands=',')
kospi_vol.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [64]:
kospi_vol.columns = ['Date','last','open','high','low','volume','%change']

kospi_vol.columns = add_feature_name(kospi_vol.columns,'kospi_vol')
# kospi_vol = kospi_vol.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

kospi_vol.head()

,Date,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,volume_kospi_vol,%change_kospi_vol
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [65]:
kospi_vol = kospi_vol[['Date', 'last_kospi_vol', 'open_kospi_vol', 'high_kospi_vol',
       'low_kospi_vol', '%change_kospi_vol']]

In [66]:
kospi_vol['%change_kospi_vol'] = kospi_vol['%change_kospi_vol'].str.replace('%','')
kospi_vol.Date = kospi_vol.Date.str.replace('년 ','')
kospi_vol.Date = kospi_vol.Date.str.replace('월 ','')
kospi_vol.Date = kospi_vol.Date.str.replace('일','')
kospi_vol.Date = pd.to_datetime(kospi_vol.Date, format='%Y %m %d')
kospi_vol.sort_values(by='Date', inplace=True)
kospi_vol.head()

,Date,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,%change_kospi_vol
1569,2013-08-06,14.98,14.79,14.98,14.61,0.40
1568,2013-08-07,15.35,15.02,15.42,14.80,2.47
1567,2013-08-08,14.96,14.84,15.42,14.77,-2.54
1566,2013-08-09,14.38,14.95,15.05,14.38,-3.88
1565,2013-08-12,14.72,14.84,15.22,14.64,2.36


# 브렌트유

In [67]:
brent_name = battery_preprocesser.get_filelist('feature_data')[12]
brent_name

'브렌트유 선물 내역.csv'

In [68]:
brent = pd.read_csv("{}".format(brent_name), thousands=',')
brent.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 31일,66.00,66.65,66.93,65.63,171.01K,-3.57%
1,2019년 12월 30일,68.44,68.20,68.99,68.16,29.42K,0.41%
2,2019년 12월 27일,68.16,67.91,68.33,67.57,112.22K,0.35%
3,2019년 12월 26일,67.92,67.27,67.99,67.22,69.82K,1.07%
4,2019년 12월 24일,67.20,66.44,67.26,66.36,104.94K,1.22%


In [69]:
brent.columns = ['Date','last','open','high','low','volume','%change']
brent.columns = add_feature_name(brent.columns,'brent')
# kospi_vol = kospi_vol.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

brent.head()

,Date,last_brent,open_brent,high_brent,low_brent,volume_brent,%change_brent
0,2019년 12월 31일,66.00,66.65,66.93,65.63,171.01K,-3.57%
1,2019년 12월 30일,68.44,68.20,68.99,68.16,29.42K,0.41%
2,2019년 12월 27일,68.16,67.91,68.33,67.57,112.22K,0.35%
3,2019년 12월 26일,67.92,67.27,67.99,67.22,69.82K,1.07%
4,2019년 12월 24일,67.20,66.44,67.26,66.36,104.94K,1.22%


In [70]:
brent['volume_brent'] = brent['volume_brent'].str.replace('-','0')
brent['volume_brent'] = brent['volume_brent'].str.replace('K','')
brent['%change_brent'] = brent['%change_brent'].str.replace('%','')
brent.Date = brent.Date.str.replace('년 ','')
brent.Date = brent.Date.str.replace('월 ','')
brent.Date = brent.Date.str.replace('일','')
brent.Date = pd.to_datetime(brent.Date, format='%Y %m %d')
brent.sort_values(by='Date', inplace=True)
brent.head()

,Date,last_brent,open_brent,high_brent,low_brent,volume_brent,%change_brent
2580,2010-01-04,80.12,78.49,80.48,78.34,122.64,2.81
2579,2010-01-05,80.59,80.29,80.84,79.75,131.75,0.59
2578,2010-01-06,81.89,80.38,82.21,79.77,157.87,1.61
2577,2010-01-07,81.51,82.00,82.05,81.05,131.28,-0.46
2576,2010-01-08,81.37,81.51,82.05,80.59,157.49,-0.17


# WTI유

In [71]:
wti_name = battery_preprocesser.get_filelist('feature_data')[11]
wti_name

'WTI유 선물 내역.csv'

In [72]:
wti = pd.read_csv("{}".format(wti_name), thousands=',')

wti.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 31일,61.06,61.68,61.88,60.63,494.54K,-1.01%
1,2019년 12월 30일,61.68,61.71,62.34,61.09,427.15K,-0.06%
2,2019년 12월 27일,61.72,61.73,61.97,61.24,351.90K,0.06%
3,2019년 12월 26일,61.68,61.20,61.83,61.06,265.09K,0.80%
4,2019년 12월 25일,61.19,61.45,61.52,61.17,-,0.13%


In [73]:
wti.columns = ['Date','last','open','high','low','volume','%change']
wti.columns = add_feature_name(wti.columns,'wti')
# kospi_vol = kospi_vol.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

wti.head()

,Date,last_wti,open_wti,high_wti,low_wti,volume_wti,%change_wti
0,2019년 12월 31일,61.06,61.68,61.88,60.63,494.54K,-1.01%
1,2019년 12월 30일,61.68,61.71,62.34,61.09,427.15K,-0.06%
2,2019년 12월 27일,61.72,61.73,61.97,61.24,351.90K,0.06%
3,2019년 12월 26일,61.68,61.20,61.83,61.06,265.09K,0.80%
4,2019년 12월 25일,61.19,61.45,61.52,61.17,-,0.13%


In [74]:
wti['volume_wti'] = wti['volume_wti'].str.replace('K','')
wti['volume_wti'] = wti['volume_wti'].str.replace('M','000')
wti['volume_wti'] = wti['volume_wti'].replace('-', np.nan)
wti['%change_wti'] = wti['%change_wti'].str.replace('%','')
wti.Date = wti.Date.str.replace('년 ','')
wti.Date = wti.Date.str.replace('월 ','')
wti.Date = wti.Date.str.replace('일','')
wti.Date = pd.to_datetime(wti.Date, format='%Y %m %d')
wti.sort_values(by='Date', inplace=True)
wti.head()

,Date,last_wti,open_wti,high_wti,low_wti,volume_wti,%change_wti
2590,2010-01-04,81.51,79.63,81.79,79.63,263.54,2.71
2589,2010-01-05,81.77,81.63,82.00,80.95,258.89,0.32
2588,2010-01-06,83.18,81.43,83.52,80.85,370.06,1.72
2587,2010-01-07,82.66,83.20,83.36,82.26,246.63,-0.63
2586,2010-01-08,82.75,82.65,83.47,81.80,310.38,0.11


In [75]:
wti.tail()

,Date,last_wti,open_wti,high_wti,low_wti,volume_wti,%change_wti
4,2019-12-25,61.19,61.45,61.52,61.17,NaN,0.13
3,2019-12-26,61.68,61.20,61.83,61.06,265.09,0.80
2,2019-12-27,61.72,61.73,61.97,61.24,351.90,0.06
1,2019-12-30,61.68,61.71,62.34,61.09,427.15,-0.06
0,2019-12-31,61.06,61.68,61.88,60.63,494.54,-1.01


# tech

In [76]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지')
data_name ='event_embbed_with_tech_{}.xlsx'.format(company)

In [77]:
tech = pd.read_excel("{}".format(data_name), thousands=',',index_col=0)
tech.tail(20)

,date,rsi_14,macd,cci,adx,stoch_slowk,stoch_slowd,willr,momentum,roc,ema20,adosc,obv,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,event11,event12,event13,event14,event15,event16
2699,2019-12-02,61.010359,2.061716,135.881271,14.136938,62.893830,61.043489,-9.090909,100.0,4.264392,2331.502065,1.184049e+05,297963937.0,0.109354,0.237200,0.190213,0.311294,0.207202,0.163864,0.238719,0.167507,0.270324,0.188469,0.100033,-0.028611,0.233231,0.035097,0.185384,0.087060
2700,2019-12-03,55.535585,6.701103,101.991614,15.083407,62.183236,60.065808,-27.272727,70.0,3.010753,2337.549488,1.955799e+05,297434081.0,-0.002144,0.188849,0.112814,0.272053,0.152331,0.106338,0.142613,0.074851,0.193095,0.064522,0.022820,-0.158047,0.179848,-0.129496,0.128358,-0.025569
2701,2019-12-04,55.961166,10.658446,101.598350,15.962271,68.518519,64.531862,-25.454545,140.0,6.194690,2343.497155,2.000630e+05,297872913.0,-0.082592,0.044785,0.015095,0.143183,0.024691,-0.018143,0.053111,-0.032347,0.101273,0.023973,-0.083066,-0.204363,0.074382,-0.134135,0.000931,-0.119829
2702,2019-12-05,56.850693,14.435185,105.308964,16.973676,62.037037,64.246264,-21.818182,185.0,8.314607,2349.830760,1.933152e+05,298446513.0,-0.141664,0.028222,-0.041390,0.109000,-0.011839,-0.057699,-0.000108,-0.081373,0.043937,-0.036072,-0.126903,-0.251805,0.023391,-0.206965,-0.035859,-0.154988
2703,2019-12-06,59.076176,19.223967,116.133942,17.979412,66.203704,65.586420,-12.727273,235.0,10.681818,2357.942116,1.883634e+05,298978319.0,-0.021280,0.160541,0.101013,0.242292,0.095077,0.032886,0.102789,0.031876,0.188829,0.037490,-0.001940,-0.204588,0.156571,-0.133200,0.088624,-0.068818
2704,2019-12-09,63.167774,26.745389,134.751773,19.688211,71.085859,66.442200,-12.121212,135.0,5.744681,2370.042867,2.545981e+05,300105392.0,-0.097240,0.064155,0.014044,0.146225,0.038511,-0.003573,0.031137,-0.040464,0.108531,-0.010116,-0.085937,-0.250203,0.074295,-0.184924,-0.001246,-0.116441
2705,2019-12-10,60.559554,30.738009,113.653137,21.274954,70.075758,69.121773,-18.181818,115.0,4.893617,2379.086403,1.486420e+05,299661279.0,0.000337,0.173312,0.120839,0.234992,0.154915,0.142465,0.152886,0.053744,0.188665,0.085636,-0.018930,-0.180991,0.169935,-0.133985,0.099812,-0.036898
2706,2019-12-11,63.025730,35.909001,114.396887,23.020521,72.935235,71.365617,-10.447761,165.0,7.081545,2390.125793,1.296687e+05,300335997.0,-0.013823,0.108415,0.068722,0.196541,0.078903,0.028980,0.119655,0.021865,0.145881,0.084919,-0.023326,-0.153332,0.126964,-0.089740,0.056035,-0.059009
2707,2019-12-12,62.326232,39.152269,123.893805,24.848788,67.222939,70.077977,-18.181818,165.0,7.096774,2399.637623,-1.019056e+04,299705805.0,-0.097345,0.007026,0.018863,0.063069,0.025476,0.017498,0.093740,-0.065902,0.101436,0.045181,-0.143175,-0.213282,0.023459,-0.135602,-0.063489,-0.088843
2708,2019-12-13,57.514221,38.455088,67.881549,25.449965,54.985177,65.047784,-36.538462,155.0,6.739130,2404.910230,-1.714077e+05,299046255.0,-0.058795,0.004592,0.027366,0.094488,0.021381,0.029407,0.115536,-0.017215,0.099786,0.082966,-0.083715,-0.133279,0.043782,-0.082671,-0.033463,-0.060661


In [78]:
tech.columns

Index(['date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16'],
      dtype='object')

In [79]:
tech.columns = ['Date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16']

# event

# ohlc

In [80]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지\tech_data')

tech_data = pd.read_excel('{}_tech.xlsx'.format(company))
tech_data = tech_data[5:]
tech_data.columns = ['Date','open','high','low','close','volumne']
tech_data['profit_rate'] = (tech_data['close'] - tech_data['open'])/tech_data['open']*100
# tech_data.profit_rate = tech_data.profit_rate.shift(-1)
tech_data.Date = pd.to_datetime(tech_data.Date)

In [81]:
tech_data.head()

,Date,open,high,low,close,volumne,profit_rate
5,2009-01-02,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-01-05,NaN,NaN,NaN,NaN,NaN,NaN
7,2009-01-06,NaN,NaN,NaN,NaN,NaN,NaN
8,2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN
9,2009-01-08,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
profit_rate = tech_data[['Date','profit_rate']]

In [83]:
# profit_rate.notnull()

# concat

In [84]:
ko_market

,Date,kospi,kosdaq
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26
...,...,...,...
2461,2019-12-23,647.62,2203.71
2462,2019-12-24,638.91,2190.08
2463,2019-12-26,652.07,2197.93
2464,2019-12-27,661.24,2204.21


In [85]:
data.drop('profit_rate',axis=1,inplace=True)

In [86]:
df_list = [data, nickel, mineral_index, co_lithum, cobalt_LME, cobalt_rotterdam, ko_market, usd_krw,
           korean_bond, kospi_vol, brent, wti,tech, profit_rate]

In [87]:
# for i in df_list:
#     i.info()

In [88]:
for list_data in df_list:
    list_data.loc[:,'Date'] = pd.to_datetime(list_data.loc[:,'Date'])
#     list_data.Date = pd.to_datetime(list_data.Date)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [89]:
for feature_data in df_list[1:]:
    data = pd.merge(data,feature_data, on ='Date',how='left')

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      2461 non-null   datetime64[ns]
 1   asset(1000)               2461 non-null   object        
 2   debt(1000)                2461 non-null   object        
 3   sales(1000)               2461 non-null   object        
 4   adjusted_price            1061 non-null   object        
 5   volumne(mean_5d)(week)    1061 non-null   object        
 6   volumne(week)             1061 non-null   object        
 7   profit(1000)              2461 non-null   object        
 8   term_profit(1000)         2461 non-null   object        
 9   gross_margin(1000)        2461 non-null   object        
 10  adjusted_BPS              2461 non-null   object        
 11  adjusted_EPS              2461 non-null   object        
 12  price_nickel        

In [91]:
# data.head()

In [92]:
data.iloc[:,2:] = data.iloc[:,2:].astype('float64')

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      2461 non-null   datetime64[ns]
 1   asset(1000)               2461 non-null   object        
 2   debt(1000)                2461 non-null   float64       
 3   sales(1000)               2461 non-null   float64       
 4   adjusted_price            1061 non-null   float64       
 5   volumne(mean_5d)(week)    1061 non-null   float64       
 6   volumne(week)             1061 non-null   float64       
 7   profit(1000)              2461 non-null   float64       
 8   term_profit(1000)         2461 non-null   float64       
 9   gross_margin(1000)        2461 non-null   float64       
 10  adjusted_BPS              2461 non-null   float64       
 11  adjusted_EPS              2461 non-null   float64       
 12  price_nickel        

- %change_korean_bond         2461 non-null object
- %change_kospi_vol           1570 non-null object
- volume_brent                2447 non-null object
- %change_brent               2447 non-null object
- volume_wti                  2431 non-null object
- %change_wti                 2431 non-null object

In [94]:
# data['volume_wti']=='1.01M'

In [95]:
# data['%change_usd_krw'] = data['%change_usd_krw'].astype('float64')
# data['%change_korean_bond'] = data['%change_korean_bond'].astype('float64')
# data['%change_kospi_vol'] = data['%change_kospi_vol'].astype('float64')
# data['volume_brent'] = data['volume_brent'].astype('float64')
# data['%change_brent'] = data['%change_brent'].astype('float64')

In [96]:
# data['volume_wti'] = data['volume_wti'].astype('float64')
# data['%change_wti'] = data['%change_wti'].astype('float64')

In [97]:

data['rate_rolling_3'] = data.profit_rate.rolling(window=3).mean()
data['rate_rolling_5'] = data.profit_rate.rolling(window=5).mean()
data['rate_rolling_10'] = data.profit_rate.rolling(window=10).mean()
data['rate_rolling_30'] = data.profit_rate.rolling(window=30).mean()
data['rate_rolling_120'] = data.profit_rate.rolling(window=120).mean()
data['rate_rolling_150'] = data.profit_rate.rolling(window=150).mean()
data['profit_rate'] = data.profit_rate.shift(periods=-1)

In [98]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지')
data.to_csv("data_1_{}.csv".format(company), encoding='utf-8')

In [99]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS,price_nickel,change_nickel,%change_nickel,stock_nickel,index_mineral,change_mineral,%change_mineral,major_mineral,major_change_mineral,major_%chagne_mineral,price_co_lithum,change_co_lithum,%change_co_lithum,price_cobalt_LME,change_cobalt_LME,%change_cobalt_LME,price_cobalt_rotterdam,change_cobalt_rotterdam,%change_cobalt_rotterdam,kospi,kosdaq,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,%change_kospi_vol,last_brent,open_brent,high_brent,low_brent,volume_brent,%change_brent,last_wti,open_wti,high_wti,low_wti,volume_wti,%change_wti,rsi_14,macd,cci,adx,stoch_slowk,stoch_slowd,willr,momentum,roc,ema20,adosc,obv,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,event11,event12,event13,event14,event15,event16,profit_rate,rate_rolling_3,rate_rolling_5,rate_rolling_10,rate_rolling_30,rate_rolling_120,rate_rolling_150
0,2010-01-11,24045185,14580313.0,28735454.0,NaN,NaN,NaN,6836614.0,5891293.0,8727267.0,222.23,138.32,18300.0,175.0,0.97,159792.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,540.40,1694.12,1120.50,1124.70,1125.4,1115.40,-0.69,4.840,4.840,4.840,4.840,-1.43,NaN,NaN,NaN,NaN,NaN,80.97,81.53,82.45,80.41,137.69,-0.49,82.52,82.88,83.95,81.96,296.30,-0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.144804,-0.039180,-0.036545,0.033332,-0.010435,-0.001088,0.022977,-0.090904,0.042332,-0.012466,-0.154154,-0.207489,-0.019969,-0.161041,-0.099533,-0.115672,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,24045185,14580313.0,28735454.0,NaN,NaN,NaN,6836614.0,5891293.0,8727267.0,222.23,138.32,17765.0,-535.0,-2.92,159726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,538.11,1698.64,1120.75,1119.70,1127.3,1117.30,0.02,4.860,4.860,4.860,4.860,0.41,NaN,NaN,NaN,NaN,NaN,79.30,80.55,80.86,78.49,149.36,-2.06,80.79,82.07,82.34,79.91,333.87,-2.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.070792,0.063178,0.028834,0.131254,0.074361,0.083712,0.101119,-0.013229,0.101959,0.043821,-0.097684,-0.171642,0.060432,-0.154362,-0.018807,-0.071966,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,24045185,14580313.0,28735454.0,NaN,NaN,NaN,6836614.0,5891293.0,8727267.0,222.23,138.32,17735.0,-30.0,-0.17,159648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,534.46,1671.41,1123.00,1120.75,1130.6,1118.80,0.20,4.820,4.820,4.820,4.820,-0.82,NaN,NaN,NaN,NaN,NaN,78.31,78.49,79.09,77.04,105.59,-1.25,79.65,80.06,80.67,78.37,401.63,-1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.073607,0.076268,0.041688,0.188937,0.074965,0.055785,0.057590,0.027561,0.148355,0.007852,-0.013603,-0.107155,0.117566,-0.091479,0.067659,-0.032910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,24045185,14580313.0,28735454.0,NaN,NaN,NaN,6836614.0,5891293.0,8727267.0,222.23,138.32,18350.0,615.0,3.47,161550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,542.33,1685.77,1123.00,1122.50,1123.9,1118.40,0.00,4.775,4.775,4.775,4.775,-0.93,NaN,NaN,NaN,NaN,NaN,77.82,78.55,78.98,77.49,22.48,-0.63,79.39,79.63,80.36,78.92,275.40,-0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.102128,0.220774,0.215662,0.298658,0.212121,0.213876,0.242790,0.143532,0.276082,0.181676,0.078864,-0.040092,0.241608,0.001869,0.179814,0.088941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,24045185,14580313.0,28735454.0,NaN,NaN,NaN,6836614.0,5891293.0,8727267.0,222.23,138.32,18325.0,-25.0,-0.14,161994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.25,1.25,5.68,549.97,1701.80,1123.50,1123.00,1125.3,1120.86,0.04,4.775,4.775,4.775,4.775,0.00,NaN,NaN,NaN,NaN,NaN,77.11,78.35,78.35,76.81,144.80,-0.91,78.00,79.20,79.31,77.70,200.56,-1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.159713,0.246575,0.238641,0.353094,0.268696